In [214]:
import pandas as pd
import numpy as np

In [215]:
data = pd.read_csv('Data/train.csv')
data_test = pd.read_csv('Data/test.csv')

In [216]:
data.pixels_y = data.pixels_y.astype('int64')

In [217]:
data_test.pixels_y = data_test.pixels_y.fillna(0)
data_test.pixels_y = data_test.pixels_y.astype('int64')

In [218]:
data.dtypes

id                       int64
name                    object
brand                   object
base_name               object
screen_size            float64
pixels_x                 int64
pixels_y                 int64
screen_surface          object
touchscreen              int64
cpu                     object
cpu_details             object
detachable_keyboard    float64
discrete_gpu             int64
gpu                     object
os                      object
os_details              object
ram                      int64
ssd                      int64
storage                  int64
weight                 float64
min_price              float64
max_price              float64
dtype: object

In [219]:
data_s = data[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu','discrete_gpu','gpu','os_details','ram','ssd','storage','weight','min_price','max_price']]
data_test_s = data_test[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu','discrete_gpu','gpu','os_details','ram','ssd','storage','weight']]

In [220]:
data_s

,id,brand,screen_size,pixels_y,screen_surface,touchscreen,cpu,discrete_gpu,gpu,os_details,ram,ssd,storage,weight,min_price,max_price
0,7774,Lenovo,15.6,1080,Glossy,1,Intel Core i7,0,Intel HD,Windows 10,8,0,1000,4.60,899.00,899.00
1,25926,Razer,15.6,1080,Matte,0,Intel Core i7,1,NVIDIA GeForce RTX 2070 Max-Q,Windows 10 Home,16,512,512,4.63,2099.99,2099.99
2,25267,HP,15.6,768,NaN,0,AMD A6,0,AMD Radeon R4,Windows 10,8,0,500,4.63,439.00,449.00
3,22367,Acer,15.6,1080,Matte,0,Intel Core i3,0,Intel UHD 620,Windows 10 Home,6,0,1000,5.30,375.00,449.00
4,17471,HP,17.3,900,Glossy,0,Intel Core i5,0,Intel HD 620,Windows 10,8,0,1000,5.80,559.00,559.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,3897,Dell,15.6,768,Glossy,1,Intel Core i3,0,Intel HD 4400,Windows 8.1,4,0,500,6.00,383.51,383.51
506,19301,Asus,17.3,1080,Matte,0,Intel Core i7,1,NVIDIA GeForce GTX 1070,Windows 10 Home,12,128,1128,6.39,1899.99,1899.99
507,22738,Asus,15.6,1080,Matte,0,Intel Core i5,0,Intel UHD 620,Windows 10 Home,8,128,1128,3.70,499.99,669.97
508,16468,HP,17.3,900,Glossy,0,Intel Core i5,0,Intel HD 620,Windows 10,8,0,1000,5.84,548.29,548.29


In [221]:
def reformat(df):
    df_pp = df.copy()
    df_pp["cpu_brand"] = df_pp.cpu.str.split(n=1).str[0]
    df_pp["cpu_type"] = df_pp.cpu.str.split(n=1).str[1]
    df_pp["gpu_brand"] = df_pp.gpu.str.split(n=1).str[0]
    df_pp["gpu_series"] = df_pp.gpu.str.split(n=2).str[1]
    df_pp["os_type"] = df_pp.os_details.str.split(n=1).str[1]
    df_pp = df_pp.drop(columns=["os_details", "cpu", "gpu", "os_details"])
    return df_pp

In [222]:
def process_onehot(df, list_values):
    transformed_df = pd.DataFrame()
    from sklearn import preprocessing
    for item in list_values:
        enc = preprocessing.OneHotEncoder()
        df[item] = df[item].astype(str)
        df[item] = df[item].str.lower()
        enc.fit(df[[item]].dropna())
        new_df = pd.DataFrame(data=enc.transform(df[[item]].dropna()).toarray(), columns=enc.categories_[0])
        transformed_df = pd.concat([transformed_df, new_df], axis=1)
    return transformed_df

In [223]:
def process_ordinal(df, list_values):
    transformed_df = pd.DataFrame()
    from sklearn import preprocessing
    for item in list_values:
        new_df = pd.DataFrame()
        enc = preprocessing.OrdinalEncoder()
        df[item] = df[item].astype(str)
        df[item] = df[item].str.lower()
        enc.fit(df[[item]].dropna())
        temp = enc.transform(df[[item]].dropna()).shape
        new_df[item] = enc.transform(df[[item]].dropna()).flatten()
        transformed_df = pd.concat([transformed_df, new_df], axis=1)
        print(new_df)
    return transformed_df

In [224]:
data_s = reformat(data_s)
data_test_s = reformat(data_test_s)

In [225]:
data_t = pd.concat([data_s, data_test_s]).reset_index(drop=True)
data_t

,id,brand,screen_size,pixels_y,screen_surface,touchscreen,discrete_gpu,ram,ssd,storage,weight,min_price,max_price,cpu_brand,cpu_type,gpu_brand,gpu_series,os_type
0,7774,Lenovo,15.6,1080,Glossy,1,0,8,0,1000,4.60,899.00,899.00,Intel,Core i7,Intel,HD,10
1,25926,Razer,15.6,1080,Matte,0,1,16,512,512,4.63,2099.99,2099.99,Intel,Core i7,NVIDIA,GeForce,10 Home
2,25267,HP,15.6,768,NaN,0,0,8,0,500,4.63,439.00,449.00,AMD,A6,AMD,Radeon,10
3,22367,Acer,15.6,1080,Matte,0,0,6,0,1000,5.30,375.00,449.00,Intel,Core i3,Intel,UHD,10 Home
4,17471,HP,17.3,900,Glossy,0,0,8,0,1000,5.80,559.00,559.00,Intel,Core i5,Intel,HD,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,8398,Asus,13.3,1080,Matte,0,0,8,256,256,2.65,NaN,NaN,Intel,Core m3,Intel,HD,10
728,21211,LG,15.6,1080,Glossy,1,0,16,1000,1000,2.41,NaN,NaN,Intel,Core i7,Intel,UHD,10 Home
729,22553,Razer,15.6,1080,Matte,0,1,16,512,512,4.63,NaN,NaN,Intel,Core i7,NVIDIA,GeForce,10 Home
730,28808,MSI,15.6,1080,Matte,0,1,8,256,256,4.10,NaN,NaN,Intel,Core i5,NVIDIA,GeForce,10 Home


In [226]:
#processed_data_total = process_onehot(data_t,['screen_surface', 'cpu_type', 'gpu_brand', "gpu_series", 'os_type','brand' ])

In [227]:
processed_data_total = process_ordinal(data_t,['screen_surface', 'cpu_type', 'gpu_brand', "gpu_series", 'os_type','brand' ])

     screen_surface
0               0.0
1               1.0
2               2.0
3               1.0
4               0.0
..              ...
727             1.0
728             0.0
729             1.0
730             1.0
731             0.0

[732 rows x 1 columns]
     cpu_type
0        10.0
1        10.0
2         3.0
3         8.0
4         9.0
..        ...
727      13.0
728      10.0
729      10.0
730       9.0
731      10.0

[732 rows x 1 columns]
     gpu_brand
0          2.0
1          7.0
2          0.0
3          2.0
4          2.0
..         ...
727        2.0
728        2.0
729        7.0
730        7.0
731        0.0

[732 rows x 1 columns]
     gpu_series
0           3.0
1           1.0
2           9.0
3          11.0
4           3.0
..          ...
727         3.0
728        11.0
729         1.0
730         1.0
731         9.0

[732 rows x 1 columns]
     os_type
0        0.0
1        1.0
2        0.0
3        1.0
4        0.0
..       ...
727      0.0
728      1.0
729    

In [247]:
#data_f = pd.concat([processed_data,data_s[['id','pixels_y','ram','ssd','storage','weight', 'discrete_gpu','touchscreen','min_price','max_price']]], axis=1).set_index('id')
#data_f_test = pd.concat([processed_data_test,data_test_s[['id','pixels_y','ram','ssd','storage','weight','touchscreen', 'discrete_gpu']]], axis=1).set_index('id').fillna(0)

In [229]:
data_t = data_t[['id','pixels_y','ram','ssd','storage','weight', 'discrete_gpu','touchscreen','min_price','max_price']]

In [230]:
processed_data_total

,screen_surface,cpu_type,gpu_brand,gpu_series,os_type,brand
0,0.0,10.0,2.0,3.0,0.0,9.0
1,1.0,10.0,7.0,1.0,1.0,14.0
2,2.0,3.0,0.0,9.0,0.0,6.0
3,1.0,8.0,2.0,11.0,1.0,0.0
4,0.0,9.0,2.0,3.0,0.0,6.0
...,...,...,...,...,...,...
727,1.0,13.0,2.0,3.0,0.0,3.0
728,0.0,10.0,2.0,11.0,1.0,10.0
729,1.0,10.0,7.0,1.0,1.0,14.0
730,1.0,9.0,7.0,1.0,1.0,12.0


In [231]:
data_t.dtypes

id                int64
pixels_y          int64
ram               int64
ssd               int64
storage           int64
weight          float64
discrete_gpu      int64
touchscreen       int64
min_price       float64
max_price       float64
dtype: object

In [232]:
data_total = pd.concat([processed_data_total, data_t], axis=1).set_index("id")

In [233]:
data_total

,screen_surface,cpu_type,gpu_brand,gpu_series,os_type,brand,pixels_y,ram,ssd,storage,weight,discrete_gpu,touchscreen,min_price,max_price
id,,,,,,,,,,,,,,,
7774,0.0,10.0,2.0,3.0,0.0,9.0,1080,8,0,1000,4.60,0,1,899.00,899.00
25926,1.0,10.0,7.0,1.0,1.0,14.0,1080,16,512,512,4.63,1,0,2099.99,2099.99
25267,2.0,3.0,0.0,9.0,0.0,6.0,768,8,0,500,4.63,0,0,439.00,449.00
22367,1.0,8.0,2.0,11.0,1.0,0.0,1080,6,0,1000,5.30,0,0,375.00,449.00
17471,0.0,9.0,2.0,3.0,0.0,6.0,900,8,0,1000,5.80,0,0,559.00,559.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8398,1.0,13.0,2.0,3.0,0.0,3.0,1080,8,256,256,2.65,0,0,NaN,NaN
21211,0.0,10.0,2.0,11.0,1.0,10.0,1080,16,1000,1000,2.41,0,1,NaN,NaN
22553,1.0,10.0,7.0,1.0,1.0,14.0,1080,16,512,512,4.63,1,0,NaN,NaN


In [248]:
data_total_train = data_total.filter(items=data.id, axis=0)
data_total_test = data_total.filter(items=data_test.id, axis=0).iloc[:,0:-2].fillna(0)

In [307]:
from sklearn import decomposition
from sklearn import model_selection
from sklearn import ensemble
from sklearn import cluster
from sklearn import svm
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import LeavePOut

In [281]:

X_1 = data_total_train.iloc[:,0:-2].fillna(0)
y_1 = data_total_train.iloc[:,-2].fillna(0)
X_2 = data_total_train.iloc[:,0:-2].fillna(0)
y_2 = data_total_train.iloc[:,-1].fillna(0)
y_2

id
7774      899.00
25926    2099.99
25267     449.00
22367     449.00
17471     559.00
          ...   
3897      383.51
19301    1899.99
22738     669.97
16468     548.29
29633    1699.00
Name: max_price, Length: 510, dtype: float64

In [282]:
#clustering = cluster.FeatureAgglomeration(n_clusters=5)
#X_1 = clustering.fit_transform(X_1)
#X_2 = clustering.fit_transform(X_2)
#data_f_test_pp = clustering.fit_transform(data_total_test)

In [283]:
#pca = decomposition.PCA(n_components=20)
#X_1 = pca.fit_transform(X_1)
#X_2 = pca.fit_transform(X_2)
#data_f_test_pp = pca.fit_transform(data_f_test_pp)

In [284]:
pp = preprocessing.PowerTransformer(standardize=True)
#X_1 = pp.fit_transform(X_1)
#X_2 = pp.fit_transform(X_2)
#data_f_test_pp = pp.fit_transform(data_f_test_pp)


In [343]:
parameters = {'n_estimators':np.linspace(5,500,10).astype(int),'max_depth':(np.linspace(1,12,5)), 'criterion':('mse','mae')}
model = ensemble.RandomForestRegressor()
model2 = ensemble.RandomForestRegressor()
lpout = LeavePOut(50)

clf = model_selection.GridSearchCV(model, parameters, cv=5, scoring='neg_median_absolute_error', n_jobs=8)
clf2 = model_selection.GridSearchCV(model2, parameters, cv=5, scoring='neg_median_absolute_error', n_jobs=8)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_1,y_1, test_size=0.3, random_state=40 )
predictions = pd.DataFrame(index=y_test.index)

clf.fit(X_train, y_train)
model = clf.best_estimator_
model.fit(X_train, y_train)
predictions['min_pred'] = model.predict(X_test)
predictions['min_true'] = y_test
predictions['min_error'] = np.abs(predictions.min_pred-predictions.min_true)



In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_2,y_2, test_size=0.3, random_state=40)
clf.fit(X_train, y_train)
model2 = clf.best_estimator_
model2.fit(X_train, y_train)
predictions['max_pred'] = model2.predict(X_test)
predictions['max_true'] = y_test
predictions['max_error'] = np.abs(predictions.max_pred-predictions.max_true)


In [ ]:
print(predictions.min_error.mean())
print(predictions.max_error.mean())

In [ ]:
np.mean(predictions.min_error+predictions.max_error)


In [ ]:
clf.best_estimator_

In [291]:
predictions = pd.DataFrame(index=data_total_test.index)
model.fit(X_1,y_1)
model2.fit(X_2,y_2)
predictions['min_pred'] = model.predict(data_total_test.fillna(0))
predictions['max_pred'] = model2.predict(data_total_test.fillna(0))

In [292]:
predictions.to_csv('results_steven.csv')

In [293]:
predictions

,min_pred,max_pred
id,,
28807,1090.435577,1180.485350
22559,363.647179,385.291550
28647,721.920192,765.381300
22141,551.428462,569.803700
26116,1221.197692,1312.951100
...,...,...
8398,964.949712,1012.051450
21211,1332.273846,1529.210900
22553,2064.203846,2205.242967


In [126]:
data_total_test.columns

Index(['screen_surface', 'cpu_type', 'gpu_brand', 'gpu_series', 'os_type',
       'brand', 'pixels_y', 'ram', 'ssd', 'storage', 'weight', 'discrete_gpu',
       'touchscreen'],
      dtype='object')

In [127]:
X_1.columns

Index(['screen_surface', 'cpu_type', 'gpu_brand', 'gpu_series', 'os_type',
       'brand', 'pixels_y', 'ram', 'ssd', 'storage', 'weight', 'discrete_gpu',
       'touchscreen'],
      dtype='object')